# TensorFlow and TensorBoard with Regularization



## Purpose

The purpose of this lab is threefold.  

1.   to review using `TensorFlow` and `TensorBoard` for modeling and evaluation with neural networks
2.   to review using data science pipelines and cross-validation with neural networks
3.   to review using `TensorFlow` for neural network regularization

We'll be continuting our investigation of the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview) that we began [previously](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [2]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from sklearn.impute import SimpleImputer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

### Loading the data

In [3]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

You have already performed EDA on this data set. Look back on what you did before or see [here](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

Of course, feel free to re-run what you have done before or try out some other EDA as you find useful.

### Preprocessing

Let's do the same prepricessing as before.

In [4]:
# Performing preprocessing on the train and test data will be more effecient if we combine the two date sets.
combined = pd.concat([train, test], axis=0, sort=False)

#Age column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age

# Embarked column
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)

# Class column
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass

# Making Age into adult (1) and child (0)
combined['Child'] = combined['Age'].apply(lambda age: 1 if age>=18 else 0) 

# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],axis=1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

As you are now using regularization to guard against high variance, i.e. overfitting the data, in the definition of the model below include *dropout* and/or *l2* regularization. Also, feel free to experiment with different activation functions.

In [7]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
# insert Sequential and layers here
    classifier = Sequential()
    classifier.add(Dense(50,input_shape=(X_train.shape[1],),activation='sigmoid'))
    classifier.add(Dense(20,activation='relu'))
    classifier.add(Dense(1,activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

#### Use grid search to find help you tune the parameters

You can play with optimizers, epochs, and batch sizes. The ones that we're suggesting are not necessarily the best.

In [8]:
# Grid Search
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

Epoch 1/10
45/45 [==============================] - 0s 2ms/step - loss: 0.7288 - accuracy: 0.5688
Epoch 2/10
45/45 [==============================] - 0s 5ms/step - loss: 0.5769 - accuracy: 0.7879
Epoch 3/10
45/45 [==============================] - 0s 3ms/step - loss: 0.5167 - accuracy: 0.7963
Epoch 4/10
45/45 [==============================] - 0s 7ms/step - loss: 0.4790 - accuracy: 0.7992
Epoch 5/10
45/45 [==============================] - 0s 8ms/step - loss: 0.4546 - accuracy: 0.8048
Epoch 6/10
45/45 [==============================] - 0s 6ms/step - loss: 0.4458 - accuracy: 0.8062
Epoch 7/10
45/45 [==============================] - 0s 5ms/step - loss: 0.4315 - accuracy: 0.8062
Epoch 8/10
45/45 [==============================] - 0s 5ms/step - loss: 0.4207 - accuracy: 0.8272
Epoch 9/10
45/45 [==============================] - 0s 7ms/step - loss: 0.4189 - accuracy: 0.8188
Epoch 10/10
45/45 [==============================] - 0s 8ms/step - loss: 0.4157 - accuracy: 0.8258
Instructions for up

45/45 [==============================] - 0s 6ms/step - loss: 0.4355 - accuracy: 0.8135
Epoch 7/20
45/45 [==============================] - 0s 4ms/step - loss: 0.4324 - accuracy: 0.8177
Epoch 8/20
45/45 [==============================] - 0s 4ms/step - loss: 0.4266 - accuracy: 0.8261
Epoch 9/20
45/45 [==============================] - 0s 5ms/step - loss: 0.4295 - accuracy: 0.8177
Epoch 10/20
45/45 [==============================] - 0s 4ms/step - loss: 0.4263 - accuracy: 0.8191
Epoch 11/20
45/45 [==============================] - 0s 2ms/step - loss: 0.4195 - accuracy: 0.8275
Epoch 12/20
45/45 [==============================] - 0s 6ms/step - loss: 0.4165 - accuracy: 0.8261
Epoch 13/20
45/45 [==============================] - 0s 5ms/step - loss: 0.4159 - accuracy: 0.8247
Epoch 14/20
45/45 [==============================] - 0s 5ms/step - loss: 0.4143 - accuracy: 0.8317
Epoch 15/20
45/45 [==============================] - 0s 5ms/step - loss: 0.4109 - accuracy: 0.8317
Epoch 16/20
45/45 [======

45/45 [==============================] - 0s 6ms/step - loss: 0.4246 - accuracy: 0.8160
Epoch 9/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4254 - accuracy: 0.8146
Epoch 10/50
45/45 [==============================] - 0s 5ms/step - loss: 0.4227 - accuracy: 0.8174
Epoch 11/50
45/45 [==============================] - 0s 4ms/step - loss: 0.4188 - accuracy: 0.8188
Epoch 12/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4231 - accuracy: 0.8272
Epoch 13/50
45/45 [==============================] - 0s 6ms/step - loss: 0.4152 - accuracy: 0.8230
Epoch 14/50
45/45 [==============================] - 0s 4ms/step - loss: 0.4139 - accuracy: 0.8230
Epoch 15/50
45/45 [==============================] - 0s 4ms/step - loss: 0.4123 - accuracy: 0.8244
Epoch 16/50
45/45 [==============================] - 0s 4ms/step - loss: 0.4098 - accuracy: 0.8244
Epoch 17/50
45/45 [==============================] - 0s 7ms/step - loss: 0.4114 - accuracy: 0.8272
Epoch 18/50
45/45 [====

45/45 [==============================] - 0s 6ms/step - loss: 0.3878 - accuracy: 0.8373
Epoch 39/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3859 - accuracy: 0.8331
Epoch 40/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3858 - accuracy: 0.8387
Epoch 41/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3841 - accuracy: 0.8345
Epoch 42/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3822 - accuracy: 0.8373
Epoch 43/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3848 - accuracy: 0.8345
Epoch 44/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3835 - accuracy: 0.8387
Epoch 45/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3830 - accuracy: 0.8401
Epoch 46/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3825 - accuracy: 0.8373
Epoch 47/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3839 - accuracy: 0.8359
Epoch 48/50
45/45 [===

Epoch 20/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3927 - accuracy: 0.8373
Epoch 21/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3930 - accuracy: 0.8373
Epoch 22/50
45/45 [==============================] - 0s 2ms/step - loss: 0.3923 - accuracy: 0.8373
Epoch 23/50
45/45 [==============================] - 0s 2ms/step - loss: 0.3904 - accuracy: 0.8471
Epoch 24/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3889 - accuracy: 0.8415
Epoch 25/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3872 - accuracy: 0.8541
Epoch 26/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3899 - accuracy: 0.8429
Epoch 27/50
45/45 [==============================] - 0s 5ms/step - loss: 0.3886 - accuracy: 0.8443
Epoch 28/50
45/45 [==============================] - 0s 3ms/step - loss: 0.3899 - accuracy: 0.8471
Epoch 29/50
45/45 [==============================] - 0s 4ms/step - loss: 0.3914 - accuracy: 0.8457
Epoch 30/5

...

23/23 [==============================] - 0s 4ms/step - loss: 0.4270 - accuracy: 0.8275
Epoch 14/20
23/23 [==============================] - 0s 4ms/step - loss: 0.4233 - accuracy: 0.8289
Epoch 15/20
23/23 [==============================] - 0s 3ms/step - loss: 0.4223 - accuracy: 0.8261
Epoch 16/20
23/23 [==============================] - 0s 6ms/step - loss: 0.4162 - accuracy: 0.8303
Epoch 17/20
23/23 [==============================] - 0s 11ms/step - loss: 0.4152 - accuracy: 0.8317
Epoch 18/20
23/23 [==============================] - 0s 10ms/step - loss: 0.4130 - accuracy: 0.8317
Epoch 19/20
23/23 [==============================] - 0s 8ms/step - loss: 0.4106 - accuracy: 0.8303
Epoch 20/20
23/23 [==============================] - 0s 8ms/step - loss: 0.4110 - accuracy: 0.8331
Epoch 1/20
23/23 [==============================] - 0s 3ms/step - loss: 0.6635 - accuracy: 0.5456
Epoch 2/20
23/23 [==============================] - 0s 3ms/step - loss: 0.5881 - accuracy: 0.7013
Epoch 3/20
23/23 [====

23/23 [==============================] - 0s 6ms/step - loss: 0.4222 - accuracy: 0.8258
Epoch 15/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4226 - accuracy: 0.8287
Epoch 16/50
23/23 [==============================] - 0s 6ms/step - loss: 0.4184 - accuracy: 0.8287
Epoch 17/50
23/23 [==============================] - 0s 6ms/step - loss: 0.4149 - accuracy: 0.8272
Epoch 18/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4149 - accuracy: 0.8301
Epoch 19/50
23/23 [==============================] - 0s 6ms/step - loss: 0.4133 - accuracy: 0.8258
Epoch 20/50
23/23 [==============================] - 0s 6ms/step - loss: 0.4117 - accuracy: 0.8287
Epoch 21/50
23/23 [==============================] - 0s 4ms/step - loss: 0.4099 - accuracy: 0.8258
Epoch 22/50
23/23 [==============================] - 0s 3ms/step - loss: 0.4086 - accuracy: 0.8357
Epoch 23/50
23/23 [==============================] - 0s 4ms/step - loss: 0.4077 - accuracy: 0.8301
Epoch 24/50
23/23 [===

23/23 [==============================] - 0s 6ms/step - loss: 0.3854 - accuracy: 0.8415
Epoch 43/50
23/23 [==============================] - 0s 6ms/step - loss: 0.3879 - accuracy: 0.8429
Epoch 44/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3853 - accuracy: 0.8415
Epoch 45/50
23/23 [==============================] - 0s 3ms/step - loss: 0.3845 - accuracy: 0.8429
Epoch 46/50
23/23 [==============================] - 0s 6ms/step - loss: 0.3852 - accuracy: 0.8429
Epoch 47/50
23/23 [==============================] - 0s 5ms/step - loss: 0.3834 - accuracy: 0.8443
Epoch 48/50
23/23 [==============================] - 0s 7ms/step - loss: 0.3848 - accuracy: 0.8401
Epoch 49/50
23/23 [==============================] - 0s 5ms/step - loss: 0.3823 - accuracy: 0.8345
Epoch 50/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3831 - accuracy: 0.8443
Epoch 1/50
23/23 [==============================] - 0s 1ms/step - loss: 0.6061 - accuracy: 0.6438
Epoch 2/50
23/23 [=====

23/23 [==============================] - 0s 6ms/step - loss: 0.3987 - accuracy: 0.8387
Epoch 24/50
23/23 [==============================] - 0s 5ms/step - loss: 0.3967 - accuracy: 0.8401
Epoch 25/50
23/23 [==============================] - 0s 5ms/step - loss: 0.3984 - accuracy: 0.8373
Epoch 26/50
23/23 [==============================] - 0s 5ms/step - loss: 0.3960 - accuracy: 0.8401
Epoch 27/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3930 - accuracy: 0.8401
Epoch 28/50
23/23 [==============================] - 0s 5ms/step - loss: 0.3953 - accuracy: 0.8373
Epoch 29/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3934 - accuracy: 0.8359
Epoch 30/50
23/23 [==============================] - 0s 7ms/step - loss: 0.3927 - accuracy: 0.8499
Epoch 31/50
23/23 [==============================] - 0s 6ms/step - loss: 0.3921 - accuracy: 0.8443
Epoch 32/50
23/23 [==============================] - 0s 2ms/step - loss: 0.3894 - accuracy: 0.8429
Epoch 33/50
23/23 [===

56/56 [==============================] - 1s 9ms/step - loss: 0.4581 - accuracy: 0.8148: 0s - loss: 0.4942 - accu
Epoch 5/50
56/56 [==============================] - 0s 9ms/step - loss: 0.4407 - accuracy: 0.8126
Epoch 6/50
56/56 [==============================] - 0s 8ms/step - loss: 0.4365 - accuracy: 0.8215
Epoch 7/50
56/56 [==============================] - 0s 8ms/step - loss: 0.4280 - accuracy: 0.8137
Epoch 8/50
56/56 [==============================] - 0s 7ms/step - loss: 0.4253 - accuracy: 0.8215
Epoch 9/50
56/56 [==============================] - 0s 6ms/step - loss: 0.4183 - accuracy: 0.8260
Epoch 10/50
56/56 [==============================] - 0s 5ms/step - loss: 0.4164 - accuracy: 0.8238
Epoch 11/50
56/56 [==============================] - 0s 5ms/step - loss: 0.4176 - accuracy: 0.8193
Epoch 12/50
56/56 [==============================] - 0s 5ms/step - loss: 0.4136 - accuracy: 0.8283
Epoch 13/50
56/56 [==============================] - 0s 5ms/step - loss: 0.4117 - accuracy: 0.8294
E

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [9]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fitting the optimal model and evaluating with `TensorBoaard`

Define the early stopping callback. Use your best values from grid serarch with `KerasClassifer` and finally fit the model.

In [10]:
# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           verbose=1, patience=5,
                           mode='min')

# Using KerasClassifier
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

# Fit the model with the tensorboard_callback
classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[early_stop, tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

Epoch 1/50
 1/56 [..............................] - ETA: 0s - loss: 0.6302 - accuracy: 0.7500WARNING:tensorflow:From C:\Users\haibwang\AppData\Local\anaconda3\envs\learn-env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
56/56 [==============================] - 1s 12ms/step - loss: 0.6287 - accuracy: 0.6723
Epoch 2/50
56/56 [==============================] - 0s 9ms/step - loss: 0.5193 - accuracy: 0.7811
Epoch 3/50
56/56 [==============================] - 1s 11ms/step - loss: 0.4711 - accuracy: 0.7969
Epoch 4/50
56/56 [==============================] - 1s 10ms/step - loss: 0.4519 - accuracy: 0.8058
Epoch 5/50
56/56 [==============================] - 0s 8ms/step - loss: 0.4422 - accuracy: 0.8103
Epoch 6/50
56/56 [==============================] - 1s 11ms/step - loss: 0.4360 - accuracy: 0.8159
Epoch 7/50
5

Epoch 25/50
56/56 [==============================] - 1s 10ms/step - loss: 0.4052 - accuracy: 0.8294
Epoch 26/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4020 - accuracy: 0.8316
Epoch 27/50
56/56 [==============================] - 0s 6ms/step - loss: 0.4002 - accuracy: 0.8373
Epoch 28/50
56/56 [==============================] - 0s 6ms/step - loss: 0.3974 - accuracy: 0.8316
Epoch 29/50
56/56 [==============================] - 0s 3ms/step - loss: 0.3981 - accuracy: 0.8316
Epoch 30/50
56/56 [==============================] - 0s 4ms/step - loss: 0.3986 - accuracy: 0.8328
Epoch 31/50
56/56 [==============================] - 0s 4ms/step - loss: 0.3955 - accuracy: 0.8406
Epoch 32/50
56/56 [==============================] - 0s 5ms/step - loss: 0.3973 - accuracy: 0.8361
Epoch 33/50
56/56 [==============================] - 0s 8ms/step - loss: 0.3965 - accuracy: 0.8294
Epoch 34/50
56/56 [==============================] - 1s 10ms/step - loss: 0.3990 - accuracy: 0.8316
Epoch 35

In [11]:
# Call TensorBoard within SaturnCloud [Comment this out if you are not in SaturnCloud]
import os
print(f"https://{os.getenv('SATURN_JUPYTER_BASE_DOMAIN')}/proxy/8000/")
%tensorboard --logdir logs/fit --port 8000 --bind_all
# This will generate a hyperlink. Click on that to open TensorBoard!
# (You'll see a 404 error below the link, just ignore that.)

# Call TensorBoard [Not in SaturnCloud]
# Uncomment the next time if you are not in SC
%tensorboard --logdir logs/fit

https://None/proxy/8000/


Reusing TensorBoard on port 8000 (pid 11648), started 0:16:14 ago. (Use '!kill 11648' to kill it.)

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 16144.

#### Results and Predictions

Calculate the predictions, save them as a csv, and print them.

In [12]:
# Your code here (use more cells if you need to)
preds = classifier.predict(test)
results = ids.assign(Survived=preds)
results['Survived'] = results['Survived'].apply(lambda row: 1 if row > 0.5 else 0)
results.to_csv('titanic_submission.csv',index=False)
results.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

How do your predictions compare to what you did last time?

Remember that your "fancier" model may be less accurate... but that is okay if that is the case since we're trying to guard against variance with regularization techniques.